# HMM with TensorFlow

In [1]:
import tensorflow as tf
import numpy as np
import sys

## Weather HMM example
Das Wetter bei deinem Übersee-Chatfreund lässt sich durch eine Markowkette $X_1,X_2,\ldots$ mit
Zustandsraum $Q=\{\texttt{sun},\texttt{rain},\texttt{storm}\}$ und Übergangsmatrix
$$A=(A[r,s])_{\scriptsize r,s \in Q} = \begin{pmatrix}
0.7 & 0.2 & 0.1\\
0.3 & 0.5 & 0.2\\
0.2 & 0.6 & 0.2\\
\end{pmatrix}
$$
beschreiben (Reihen und Spalten in der Reihenfolge `sun`, `rain`, `storm`).

Dabei sei $X_i$ das Wetter am $i$-ten Tag und $X_1=\texttt{sun}$. Dein Freund verfolgt vom Wetter abhängig Aktivitäten entweder drinnen 
(`in`) oder draußen (`out`). Sei $\Sigma:=\{\texttt{in},\texttt{out}\}$. Folgende Matrix
beschreibe die vom Wetter abhängenden Wahrscheinlichkeiten (Wkeiten) der Aktivitäten
$$B=\big(B[q,s]\big)_{\scriptsize\begin{array}{l}q\in Q\\s\in \Sigma\end{array}} = \begin{pmatrix}
0.4 & 0.6 \\
0.8 & 0.2 \\
0.9 & 0.1 \\
\end{pmatrix}
.$$ 
Ablesebeispiel: Dein Freund bleibt mit Wkeit 0.9 drinnen, wenn es an dem Tag stürmt (Spalten in der Reihenfolge `in`, `out`).
Beantworte folgende Fragen für das durch $Q,\Sigma,A,B$ und $X_1$ gegebene Hidden-Markow-Modell.
Was ist die Wkeit 
$$P(Y_1=Y_2=Y_3=\texttt{in}),$$

dass dein Freund am allen drei Tagen drinnen bleibt?

![forward DP table](forwardManually.png)

**Solution: P(Y=y) = 0.1308**

## Specify the Model

### Example Model Parameters

In [2]:
n = 3 # number of states
s = 2 # emission alphabet size

In [3]:
A_init = np.array([[7, 2, 1], [3, 5, 2], [2, 6, 2]]) / 10.0
B_init = np.array([[4, 6], [8, 2], [9, 1]]) / 10.0
X1_dist = np.array([1., 0., 0.]) # starts with sun
n, s = B_init.shape # number of states, emission alphabet size
y = np.array([0, 0, 0]) # in, in, in

In [4]:
print("transitions:\n", A_init, "\nemissions:\n", B_init)

transitions:
 [[0.7 0.2 0.1]
 [0.3 0.5 0.2]
 [0.2 0.6 0.2]] 
emissions:
 [[0.4 0.6]
 [0.8 0.2]
 [0.9 0.1]]


#### Forward recursion
$$\alpha[i,q] = B[q, y[i]] \sum_{q'} \alpha[i-1, q'] \cdot A[q',q] $$

In [5]:
# tf variants of the transition and emission matrix
A = tf.Variable(A_init, trainable = True)
B = tf.Variable(B_init, trainable = True)

### Forward Variables and Algorithm
$$ \alpha(q,i) = \sum_{x_1,\ldots, x_{i-1}\in Q} P(x_1,\ldots, x_{i-1}, X_i=q, y_1,\ldots, y_i)$$
Initialization: 
$$ \alpha(q, 1) = \sum_{q\in Q} P(X_1 = q)\cdot B[q,y[0]]$$

In [6]:
def forward(y # observation sequence
           ):
    """ Forward Algorithm for Computing Sequence Likelihood """
    ell = y.shape[0]
    α = tf.Variable(np.zeros([ell, n]), trainable = False)
    
    # initialization
    α[0].assign(tf.multiply(B[:, y[0]], X1_dist))
    
    # forward algorithm
    for i in range(1, ell):
        # compute i-th row of DP table
        R = tf.linalg.matvec(A, α[i-1], transpose_a = True)
        α[i].assign(tf.multiply(B[:, y[i]], R))
    return α

def emiProb(α):
    return np.sum(α[-1,:])

In [7]:
α = forward(y)
α.numpy()

array([[0.4    , 0.     , 0.     ],
       [0.112  , 0.064  , 0.036  ],
       [0.04192, 0.0608 , 0.02808]])

In [8]:
Py = emiProb(α)
Py

0.1308

## A HMM as a Special Case of a Recurrent Neural Network
We use the notation of RNNs similar to that in [Dive into Deep Learning](https://d2l.ai/chapter_recurrent-neural-networks/bptt.html). $h_t$ is a size $n$ vector of RNN-"hidden states" (these are real numbers, not to be confused with the hidden states of HMMs, which are from $Q$).  
$$ h_t = f(x_t, h_{t-1}; A, B)$$
We chose the outputs
$$ o_t = \text{sum}(h_t) = h_t[0] + \cdots + h_t[n-1] \in [0,1]$$
so that the final output $o_T$ is just the likelihood of the sequence $P(Y)$.
This RNN does not need to produce intermediate outputs $o_t$ for $t<T$ as they are not used yet. However, they could be used in conjunction with a backwards pass.

### HMMCell
As a template we use the code for [tf.keras.layers.SimpleRNNCell](https://github.com/tensorflow/tensorflow/blob/v2.4.1/tensorflow/python/keras/layers/recurrent.py#L1222-L1420)

In [9]:
from HMMCell import HMMCell

## Test the HMM cell

In [10]:
print (f"n={n}, s={s}")

A_init = np.array([[[7, 2, 1], [3, 5, 2], [2, 6, 2]]]) / 10.0
B_init = np.array([[[4, 6], [8, 2], [9, 1]]]) / 10.0
I_init = np.array([[1, 1e-10, 1e-10]]) # start with X1=sun (very likely)

# take ln to cancel out the softmax that is applied to obtain a stochastic matrix 
A_init = np.log(A_init)
B_init = np.log(B_init)
I_init = np.log(I_init)

A_initializer = tf.keras.initializers.Constant(A_init)
B_initializer = tf.keras.initializers.Constant(B_init)
I_initializer = tf.keras.initializers.Constant(I_init)

yi = np.array([[1., 0]]).astype(np.float32) # np.random.random([batch_size, s]).astype(np.float32)
states = np.array([[[.4, 0, 0]]]).astype(np.float32) # np.random.random([batch_size, n]).astype(np.float32)
hmmC = HMMCell(1, n,
               transition_initializer=A_initializer,
               emission_initializer=B_initializer,
               init_initializer=I_initializer
              )

output = hmmC(yi, [0, states, [0.]])
print("output:\n", output[0], "\n")

A = hmmC.A
B = hmmC.B
I = hmmC.I

with np.printoptions(precision=5, suppress=True):
    print("transition matrix A:\n", A.numpy())
    print("emission matrix B:\n", B.numpy())
    print("initial distribution I:\n", I.numpy())


n=3, s=2
output:
 [[<tf.Tensor: shape=(1,), dtype=int8, numpy=array([0], dtype=int8)>, <tf.Tensor: shape=(1, 1, 3), dtype=float32, numpy=array([[[0.5283019 , 0.3018868 , 0.16981131]]], dtype=float32)>, <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-1.551169]], dtype=float32)>]] 

transition matrix A:
 [[[0.7 0.2 0.1]
  [0.3 0.5 0.2]
  [0.2 0.6 0.2]]]
emission matrix B:
 [[[0.4 0.6]
  [0.8 0.2]
  [0.9 0.1]]]
initial distribution I:
 [[1. 0. 0.]]


In [11]:
inputs = np.array([[[1, 0],[1, 0],[1, 0]]]).astype(np.float32)
hmm = tf.keras.layers.RNN(hmmC, return_sequences = True, return_state = True)
  
alpha, _, lastcol, loglik = hmm(inputs)
alpha = alpha[1]

print()
with np.printoptions(precision=5, suppress=True):
    print ("α=\n", alpha.numpy(),
       "\nlast column of forward table:", lastcol.numpy(),
      "\nlog-likelihood=", loglik.numpy())


α=
 [[[[1.      0.      0.     ]]

  [[0.5283  0.30189 0.16981]]

  [[0.32049 0.46483 0.21468]]]] 
last column of forward table: [[[0.32049 0.46483 0.21468]]] 
log-likelihood= [[-2.03409]]


### Training

In [12]:
from dishonest_casino import get_casino_dataset
batch_size = 32
ds = get_casino_dataset().repeat().batch(batch_size)

for inputs in ds.take(1):
    pass
inputs.shape

TensorShape([32, 100, 6])

In [13]:
n=2
u=2
dcc = HMMCell(u,n)
Q = dcc(inputs[:,0,:], [tf.zeros(batch_size, dtype=tf.int8),
                    tf.ones([batch_size, u, s], dtype=tf.float32), 
                    tf.zeros([batch_size, u], dtype=tf.float32)])

In [14]:
# the model
F = tf.keras.layers.RNN(dcc, return_state = True)

In [15]:
def print_pars(cell):
    with np.printoptions(precision=5, suppress=True):
        print("transition matrices A:\n", cell.A.numpy())
        print("emission matrices B:\n", cell.B.numpy())
        print("initial distributions I:\n", cell.I.numpy())

In [16]:
print_pars(dcc)

transition matrices A:
 [[[0.47595 0.52405]
  [0.50636 0.49364]]

 [[0.50539 0.49461]
  [0.51257 0.48743]]]
emission matrices B:
 [[[0.17278 0.16369 0.16078 0.16603 0.16941 0.16732]
  [0.16977 0.17147 0.15949 0.16716 0.16677 0.16534]]

 [[0.15911 0.17146 0.17244 0.16072 0.16531 0.17096]
  [0.16709 0.16701 0.16473 0.17497 0.159   0.16719]]]
initial distributions I:
 [[0.49281 0.50719]
 [0.48817 0.51183]]


In [17]:
def loss(model, y):
  alpha, _, lastcol, loglik = model(y)
  L = -tf.reduce_mean(loglik)
  return L

L = loss(F, inputs)
#print(f"likelihoods = {lik}\nloss (avg neg log lik)= {loss}")
print("Loss test: {}".format(L))

Loss test: 179.12921142578125


In [18]:
def grad(model, inputs):
  with tf.GradientTape() as tape:
    loss_value = loss(model, inputs)
  return loss_value, tape.gradient(loss_value, model.trainable_variables)

In [19]:
opt = tf.keras.optimizers.Adam(learning_rate=0.01)

In [20]:
model = F
loss_value, grads = grad(model, inputs)

print("Step: {}, Initial Loss: {}".format(opt.iterations.numpy(),
                                          loss_value.numpy()))


Step: 0, Initial Loss: 179.12921142578125


In [21]:
# Keep results for plotting
train_loss_results = []

num_epochs = 51
m = 20 # training batches

for epoch in range(num_epochs):
  epoch_loss_avg = tf.keras.metrics.Mean()
  epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

  # Training loop - using batches
  for y in ds.take(m):
    # Optimize the model
    loss_value, grads = grad(model, y)
    opt.apply_gradients(zip(grads, model.trainable_variables))

    # Track progress
    epoch_loss_avg.update_state(loss_value)  # Add current batch loss

    # End epoch
  train_loss_results.append(epoch_loss_avg.result())

  if epoch % 2 == 0:
    print("Epoch {:03d}: Loss: {:.3f}".format(epoch, epoch_loss_avg.result()))
  if epoch % 10 == 0:
    print_pars(dcc)

Epoch 000: Loss: 178.150
transition matrices A:
 [[[0.56331 0.43669]
  [0.59109 0.40891]]

 [[0.5839  0.4161 ]
  [0.58814 0.41186]]]
emission matrices B:
 [[[0.15836 0.15185 0.15454 0.15488 0.15917 0.22122]
  [0.15617 0.15932 0.15366 0.15663 0.15701 0.21721]]

 [[0.15054 0.15757 0.15922 0.14934 0.1588  0.22454]
  [0.15878 0.15385 0.15264 0.16323 0.15306 0.21844]]]
initial distributions I:
 [[0.46905 0.53095]
 [0.46379 0.53621]]
Epoch 002: Loss: 177.186
Epoch 004: Loss: 177.240
Epoch 006: Loss: 177.115
Epoch 008: Loss: 177.221
Epoch 010: Loss: 177.295
transition matrices A:
 [[[0.67023 0.32977]
  [0.51555 0.48445]]

 [[0.65858 0.34142]
  [0.53839 0.46161]]]
emission matrices B:
 [[[0.14986 0.14704 0.14789 0.15096 0.15049 0.25376]
  [0.15246 0.16035 0.15651 0.15136 0.15427 0.22505]]

 [[0.14681 0.15078 0.15062 0.14705 0.15165 0.25309]
  [0.15752 0.1542  0.15177 0.15757 0.1526  0.22633]]]
initial distributions I:
 [[0.07118 0.92882]
 [0.16596 0.83404]]
Epoch 012: Loss: 177.271
Epoch 014: 

In [22]:
print_pars(dcc)

transition matrices A:
 [[[0.85467 0.14533]
  [0.05003 0.94997]]

 [[0.85475 0.14525]
  [0.05007 0.94993]]]
emission matrices B:
 [[[0.1009  0.10024 0.10161 0.09938 0.09922 0.49865]
  [0.172   0.15808 0.16573 0.16604 0.16501 0.17314]]

 [[0.10097 0.10033 0.10158 0.09944 0.09929 0.49839]
  [0.17207 0.15801 0.16574 0.16604 0.16505 0.1731 ]]]
initial distributions I:
 [[0.01072 0.98928]
 [0.0123  0.9877 ]]
